In [1]:
import sys
import os
import time
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import server.services.ccxt_service  as ccxt_service
from server.services.ccxt_service import  _index_matrix,_get_nearest_vector,_create_embedding ,fetch_bulk_update
from server.services.coinmaketcap_service import   *  

from server.services import state
import asyncio
import redis.asyncio as redis
import json
import numpy as np
import faiss

In [25]:
# Fetch the data asynchronously
await fetch_bulk_update("1d",2000,False)
ohlcv_data =ccxt_service.ohlcv_data
ohlcv_symbols = ccxt_service.ohlcv_symbols
# Assuming `ohlcv_data` is populated by `fetch_bulk_update`, access the last 5 rows
ohlcv_data.tail(5)

timestamp   open   high    low  close     volume  \
tf symbol                                                                    
1d ACM/USDT  2024-10-07 07:00 +0700  1.437  1.820  1.434  1.682  2402781.0   
   ACM/USDT  2024-10-08 07:00 +0700  1.683  1.818  1.485  1.520  3954739.0   
   ACM/USDT  2024-10-09 07:00 +0700  1.520  1.650  1.490  1.540  1871423.3   
   ACM/USDT  2024-10-10 07:00 +0700  1.544  1.789  1.515  1.636  2879067.8   
   ACM/USDT  2024-10-11 07:00 +0700  1.637  1.695  1.561  1.609   929802.0   

                ema10  sma20  sma50  sma200  adx  di+  di-  
tf symbol                                                   
1d ACM/USDT  1.481545    NaN    NaN     NaN  NaN  NaN  NaN  
   ACM/USDT  1.488537    NaN    NaN     NaN  NaN  NaN  NaN  
   ACM/USDT  1.497894    NaN    NaN     NaN  NaN  NaN  NaN  
   ACM/USDT  1.523004    NaN    NaN     NaN  NaN  NaN  NaN  
   ACM/USDT  1.538640    NaN    NaN     NaN  NaN  NaN  NaN

In [ ]:
#get dataframe
symbol = "ETH/USDT"
tf = "1d"
# ohlcv_data.reset_index()
# ohlcv_data.set_index(["tf", "symbol"], inplace=True)
# symbol_df = ohlcv_data.loc[(tf, symbol)]
# symbol_df = ohlcv_data[(ohlcv_data["tf"] == tf) & (ohlcv_data["symbol"] == symbol)] #~0.4~0.6
symbol_df = ohlcv_data.query("tf == @tf and symbol == @symbol") #~0.2~0.3
symbol_df.tail(5)

KeyError: "None of ['tf', 'symbol'] are in the columns"

In [4]:
# Create embedding for the symbol
window =30
embedding = _create_embedding(symbol_df,window)
embedding.tail(5)

,open,high,low,close,volume,ema10,sma20,sma50,sma200,adx,di+,di-,position
2548067,0.858480,0.872788,0.784488,0.808115,0.100146,0.810842,0.800509,0.551671,0.087087,0.264776,0.221397,0.151317,0.862069
2548068,0.808115,0.853188,0.766744,0.803963,0.111176,0.809591,0.813009,0.557394,0.091409,0.256370,0.207925,0.154600,0.896552
2548069,0.803963,0.835272,0.723775,0.799852,0.115385,0.807820,0.816415,0.563025,0.095734,0.241810,0.191720,0.172586,0.931034
2548070,0.799851,0.840127,0.712750,0.722579,0.128731,0.792322,0.813133,0.568325,0.099682,0.226615,0.174946,0.165059,0.965517
2548071,0.722578,0.733294,0.661240,0.679732,0.075137,0.771851,0.807909,0.574980,0.103418,0.215758,0.166093,0.192881,1.000000


In [5]:
vec = embedding.values.flatten().astype("float32")
vec

array([4.65233892e-01, 5.27745187e-01, 4.53354210e-01, 4.92596328e-01,
       9.15692896e-02, 4.09133285e-01, 3.55361521e-01, 3.68487060e-01,
       0.00000000e+00, 1.17659591e-01, 2.26560846e-01, 1.91849753e-01,
       0.00000000e+00, 4.92595941e-01, 4.99400169e-01, 4.05513972e-01,
       4.33027118e-01, 7.98174068e-02, 4.13477600e-01, 3.60143304e-01,
       3.71628672e-01, 3.58332938e-04, 1.09580688e-01, 2.11306751e-01,
       2.13240564e-01, 3.44827585e-02, 4.33027506e-01, 4.49266165e-01,
       4.24543977e-01, 4.40987349e-01, 4.26485436e-03, 4.18479383e-01,
       3.64989311e-01, 3.75152647e-01, 7.42495293e-04, 1.02078855e-01,
       2.07347885e-01, 2.09245458e-01, 6.89655170e-02, 4.40987349e-01,
       4.96884584e-01, 4.26170409e-01, 4.78418440e-01, 2.96834838e-02,
       4.29377377e-01, 3.69472831e-01, 3.79676282e-01, 2.41728197e-03,
       1.00584671e-01, 2.32776508e-01, 1.97828487e-01, 1.03448279e-01,
       4.78418797e-01, 4.96884584e-01, 4.15495872e-01, 4.43399847e-01,
      

In [27]:
# indexing
   
matrix_symbols = []
vectors = []

for symbol in ohlcv_symbols[0:1000]:
    # symbol_df = ohlcv_data[(ohlcv_data["tf"] == tf) & (ohlcv_data["symbol"] == symbol)]
    symbol_df = ohlcv_data.query("tf == @tf and symbol == @symbol")
    vec = embedding.values.flatten().astype("float32")
    matrix_symbols.append(symbol)
    vectors.append(vec)
matrix = np.stack(vectors).astype("float32")
index = faiss.IndexFlatL2(matrix.shape[1])
index.add(matrix)    
index
        # spinner.stop_task(t)

KeyboardInterrupt: 

In [29]:
   
matrix_symbols = []
vectors = []

import concurrent.futures

def process_symbol(symbol):
    symbol_df = ohlcv_data.query("tf == @tf and symbol == @symbol")
    vec = embedding.values.flatten().astype("float32")
    return symbol, vec

with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    results = list(executor.map(process_symbol, ohlcv_symbols[0:1000]))

for symbol, vec in results:
    matrix_symbols.append(symbol)
    vectors.append(vec)
matrix = np.stack(vectors).astype("float32")
index = faiss.IndexFlatL2(matrix.shape[1])
index.add(matrix)    
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000020601E86E20> >